<a href="https://colab.research.google.com/github/BobCN2017/Translate/blob/master/%E2%80%9Ctransformer_translate_tf1_15_fix_ipynb%E2%80%9DTpu%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [7]:
from google.colab import drive
#新notebook中使用需要授权
drive.mount('/content/drive')
path = "/content/drive/My Drive/translate"
os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['“transformer_translate_tf1_15_fix_ipynb”Tpu版.ipynb', 'data']

In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

!pip install tensorflow-datasets==1.3.0
import tensorflow_datasets as tfds

import os
import re
import numpy as np

import matplotlib.pyplot as plt

     |████████████████████████████████| 2.4MB 3.5MB/s 
  Found existing installation: tensorflow-datasets 1.3.2
    Uninstalling tensorflow-datasets-1.3.2:
      Successfully uninstalled tensorflow-datasets-1.3.2


In [5]:
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)


INFO:tensorflow:Initializing the TPU system: 10.83.139.130:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.83.139.130:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8136483491608166466)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13730495983243268496)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10402015307010821844)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10982199782541941373)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/d

In [0]:
# Maximum number of samples to preprocess
MAX_SAMPLES = 50001

def preprocess_english_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    # adding a start and an end token to the sentence
    return sentence

def preprocess_ch(sentence):
    ret = []
    for char in sentence:
        if char == " " or char == "\t": continue
        ret.append(char)
        ret.append(" ")
    s = "".join(ret)
    return s.strip()

def load_conversations(path_resource, path_dest):
    with open(path_resource, errors='ignore', encoding="utf8") as file:
        resource = file.readlines()
    with open(path_dest, errors='ignore', encoding="utf8") as file:
        dest = file.readlines()
    inputs, outputs = [], []
    for input, output in zip(resource, dest):
        inputs.append(preprocess_english_sentence(input))
        outputs.append(preprocess_ch(output))
        if len(inputs) >= MAX_SAMPLES:
            return inputs, outputs
    return inputs, outputs


resource, dest = load_conversations("data/test.en", "data/test.zh")


In [10]:
print('Sample question: {}'.format(resource[-1]))
print('Sample answer: {}'.format(dest[-1]))
print("len of resource:{} dest:{}".format(len(resource),len(dest)))

Sample question: a couple of kids fishing poles .
Sample answer: 一 对 儿 童 鱼 竿 。
len of resource:38 dest:38


In [0]:

# Build tokenizer using tfds for both questions and answers
tokenizer_resource = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    resource, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN_RESOURCE, END_TOKEN_RESOURCE = [tokenizer_resource.vocab_size], [tokenizer_resource.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE_RESOURCE = tokenizer_resource.vocab_size + 2

# Build tokenizer using tfds for both questions and answers
tokenizer_dest = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    dest, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN_DEST, END_TOKEN_DEST = [tokenizer_dest.vocab_size], [tokenizer_dest.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE_DEST = tokenizer_dest.vocab_size + 2

In [12]:
print('Tokenized sample question: {}'.format(tokenizer_resource.encode(resource[-1])))
print('Tokenized sample dest: {}'.format(tokenizer_dest.encode(dest[-1])))

Tokenized sample question: [2, 5, 3, 120, 142, 90, 1]
Tokenized sample dest: [1, 2, 10, 117, 77, 116, 3]


In [0]:
# Maximum sentence length
MAX_LENGTH_RESOURCE = 60
MAX_LENGTH_DEST = 30

# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN_RESOURCE + tokenizer_resource.encode(sentence1) + END_TOKEN_RESOURCE
    sentence2 = START_TOKEN_DEST + tokenizer_dest.encode(sentence2) + END_TOKEN_DEST
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH_RESOURCE and len(sentence2) <= MAX_LENGTH_DEST:
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH_RESOURCE, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH_DEST, padding='post')
  
  return tokenized_inputs, tokenized_outputs


resource, dest = tokenize_and_filter(resource, dest)

In [14]:
print('Vocab size: {} dest: {}'.format(VOCAB_SIZE_RESOURCE,VOCAB_SIZE_DEST))
print(tokenizer_resource.subwords)
print(tokenizer_dest.subwords)
print('Number of samples: {}'.format(len(resource)))
print('Number of answers: {}'.format(len(dest)))

Vocab size: 454 dest: 525
[' .', 'a_', 'of_', 'the_', 'couple_', ' , ', 'pair_', 'on_', 'in_', 's_', 'couple', ' ?', 'two_', 'to_', 'this_', ' ? ', ' . ', 'what', 'their_', 'one_', 'one', 'like_', 'i_', 'had_', 'for_', ' ,', 'with_', 'winner', 'who_', 'russian_', 'rifles', 'out_', 'nines', 'mortars', 'married_', 'look_', 'just_', 'fools_', 'five_', 'crates_', 'come_', 'around_', 'and_', 'an_', 'american_', 'young_', 'you_', 'you', 'yeah', 'wusses', 'worth_', 'will_', 'what_', 'went_', 'we_', 'washed_', 'walker_', 'up_', 'two', 'townhouse', 'tonight', 'think_', 'they_', 'that_', 'territory', 't_', 'successful_', 'stealing_', 'starting_', 'star_', 'staked_', 'sitting_', 'sisters', 'sister_', 'side', 'should_', 'seen_', 'seal_', 'scoops_', 'satisfy_', 'roof_', 'road_', 'rich_', 'republican_', 'relationship', 'red_', 'pushed_', 'pseudo_', 'proof_', 'poles', 'people_', 'pecan_', 'over_', 'our_', 'or_', 'open_', 'off_', 'off', 'odds', 'not_', 'never_', 'nesting_', 'nest_', 'neat_', 'nand_', 

### Create `tf.data.Dataset`

We are going to use the [tf.data.Dataset API](https://www.tensorflow.org/api_docs/python/tf/data) to contruct our input pipline in order to utilize features like caching and prefetching to speed up the training process.

The transformer is an auto-regressive model: it makes predictions one part at a time, and uses its output so far to decide what to do next.

During training this example uses teacher-forcing. Teacher forcing is passing the true output to the next time step regardless of what the model predicts at the current time step.

As the transformer predicts each word, self-attention allows it to look at the previous words in the input sequence to better predict the next word.

To prevent the model from peaking at the expected output the model uses a look-ahead mask.

Target is divided into `decoder_inputs` which padded as an input to the decoder and `cropped_targets` for calculating our loss and accuracy.

In [0]:
BATCH_SIZE = 128
BUFFER_SIZE = 20000

# decoder inputs use the previous target as input
# remove START_TOKEN from targets

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': resource,
        'dec_inputs': dest[:, :-1]
    },
    {
        'outputs': dest[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.repeat()
dataset = dataset.batch(BATCH_SIZE)
# dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
print(dataset)

<DatasetV1Adapter shapes: ({inputs: (?, 60), dec_inputs: (?, 29)}, {outputs: (?, 29)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>


## Attention



### Scaled dot product Attention

The scaled dot-product attention function used by the transformer takes three inputs: Q (query), K (key), V (value). The equation used to calculate the attention weights is:

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

As the softmax normalization is done on the `key`, its values decide the amount of importance given to the `query`.

The output represents the multiplication of the attention weights and the `value` vector. This ensures that the words we want to focus on are kept as is and the irrelevant words are flushed out.

The dot-product attention is scaled by a factor of square root of the depth. This is done because for large values of depth, the dot product grows large in magnitude pushing the softmax function where it has small gradients resulting in a very hard softmax. 

For example, consider that `query` and `key` have a mean of 0 and variance of 1. Their matrix multiplication will have a mean of 0 and variance of `dk`. Hence, *square root of `dk`* is used for scaling (and not any other number) because the matmul of `query` and `key` should have a mean of 0 and variance of 1, so that we get a gentler softmax.

The mask is multiplied with *-1e9 (close to negative infinity).* This is done because the mask is summed with the scaled matrix multiplication of `query` and `key` and is applied immediately before a softmax. The goal is to zero out these cells, and large negative inputs to softmax are near zero in the output.

In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output

### Multi-head attention

<img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width="500" alt="multi-head attention">


Multi-head attention consists of four parts:
* Linear layers and split into heads.
* Scaled dot-product attention.
* Concatenation of heads.
* Final linear layer.

Each multi-head attention block gets three inputs; Q (query), K (key), V (value). These are put through linear (Dense) layers and split up into multiple heads. 

The `scaled_dot_product_attention` defined above is applied to each head (broadcasted for efficiency). An appropriate mask must be used in the attention step.  The attention output for each head is then concatenated (using `tf.transpose`, and `tf.reshape`) and put through a final `Dense` layer.

Instead of one single attention head, `query`, `key`, and `value` are split into multiple heads because it allows the model to jointly attend to information at different positions from different representational spaces. After the split each head has a reduced dimensionality, so the total computation cost is the same as a single head attention with full dimensionality.

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

  def get_config(self):
    return {   "num_heads":self.num_heads,
            "d_model":self.d_model,
            "depth":self.depth,
            "query_dense":self.query_dense,
            "key_dense":self.key_dense,
            "value_dense":self.value_dense,
            "dense":self.dense}

In [0]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [0]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

  def get_config(self):
    return {'pos_encoding': self.pos_encoding}

In [0]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Encoder

The Encoder consists of:
1.   Input Embedding
2.   Positional Encoding
3.   `num_layers` encoder layers

The input is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the encoder layers. The output of the encoder is the input to the decoder.

In [0]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Decoder Layer

Each decoder layer consists of sublayers:

1.   Masked multi-head attention (with look ahead mask and padding mask)
2.   Multi-head attention (with padding mask). `value` and `key` receive the *encoder output* as inputs. `query` receives the *output from the masked multi-head attention sublayer.*
3.   2 dense layers followed by dropout

Each of these sublayers has a residual connection around it followed by a layer normalization. The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis.

As `query` receives the output from decoder's first attention block, and `key` receives the encoder output, the attention weights represent the importance given to the decoder's input based on the encoder's output. In other words, the decoder predicts the next word by looking at the encoder output and self-attending to its own output. See the demonstration above in the scaled dot product attention section.

In [0]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### Decoder

The Decoder consists of:
1.   Output Embedding
2.   Positional Encoding
3.   N decoder layers

The target is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the decoder layers. The output of the decoder is the input to the final linear layer.

In [0]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### Transformer

Transformer consists of the encoder, decoder and a final linear layer. The output of the decoder is the input to the linear layer and its output is returned.

In [0]:
def transformer(vocab_size_resource,
                vocab_size_dest,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # mask the future tokens for decoder inputs at the 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # mask the encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size_resource,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size_dest,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size_dest, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

### Initialize model

To keep this example small and relatively fast, the values for *num_layers, d_model, and units* have been reduced. See the [paper](https://arxiv.org/abs/1706.03762) for all the other versions of the transformer.

In [0]:
# tf.keras.backend.clear_session()

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1



### Loss function

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [0]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH_DEST - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### Custom learning rate

Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
# sample_learning_rate = CustomSchedule(d_model=128)

# plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
# plt.ylabel("Learning Rate")
# plt.xlabel("Train Step")

### Compile Model


In [0]:
with strategy.scope():

    learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH_DEST - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
    model = transformer(
        vocab_size_resource=VOCAB_SIZE_RESOURCE,
        vocab_size_dest=VOCAB_SIZE_DEST,
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

### Fit model

Train our transformer by simply calling `model.fit()`

In [37]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Model)                 (None, None, 256)    1170432     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [38]:
EPOCHS = 2
steps_per_epoch = MAX_SAMPLES//BATCH_SIZE

model.fit(dataset,steps_per_epoch=steps_per_epoch,epochs=EPOCHS)

Epoch 1/2
390/390 [==============================] - 20s 52ms/step - loss: 1.4159 - accuracy: 0.2855
Epoch 2/2
390/390 [==============================] - 13s 32ms/step - loss: 0.0093 - accuracy: 0.4946


## Evaluate and predict

The following steps are used for evaluation:

* Apply the same preprocessing method we used to create our dataset for the input sentence.
* Tokenize the input sentence and add `START_TOKEN` and `END_TOKEN`. 
* Calculate the padding masks and the look ahead masks.
* The decoder then outputs the predictions by looking at the encoder output and its own output.
* Select the last word and calculate the argmax of that.
* Concatentate the predicted word to the decoder input as pass it to the decoder.
* In this approach, the decoder predicts the next word based on the previous words it predicted.

Note: The model used here has less capacity and trained on a subset of the full dataset, hence its performance can be further improved.

In [0]:
import time

def evaluate(sentence):
    sentence = preprocess_english_sentence(sentence)

    sentence = tf.expand_dims(
        START_TOKEN_RESOURCE + tokenizer_resource.encode(sentence) + END_TOKEN_RESOURCE, axis=0)

    output = tf.expand_dims(START_TOKEN_DEST, 0)
    start = time.time()
    for i in range(MAX_LENGTH_DEST):
        predictions = model(inputs=[sentence, output], training=False)
        print("predictions time:", time.time() - start)
        start = time.time()
        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        # if tf.equal(predicted_id, END_TOKEN[0]):
        #   break

        # concatenated the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)
        print("others time:", time.time() - start)
        start = time.time()
    return tf.squeeze(output, axis=0)


def predict(sentence):
    start = time.time()
    prediction = evaluate(sentence)
    print("prediction:", prediction)
    print("others time:", time.time() - start)
    start = time.time()
    sess = tf.keras.backend.get_session()
    print("get session time:", time.time() - start)
    start = time.time()
    ids = prediction.eval(session=sess)
    print("eval time 1:", time.time() - start)
    start = time.time()
    ids = [i for i in ids]
    print("eval time 2:", time.time() - start)
    start = time.time()
    if END_TOKEN_DEST[0] in ids:
        index = ids.index(END_TOKEN_DEST[0])
        ids = ids[:index]
    ids = [i for i in ids if i < tokenizer_dest.vocab_size]

    print("get ids time:", time.time() - start)
    start = time.time()
    predicted_sentence = tokenizer_dest.decode(ids)
    print("tokenizer.decode time:", time.time() - start)
    start = time.time()
    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence


Let's test our model!

In [40]:

output = predict('In my opinion , this is very smart.')

predictions time: 0.6424820423126221
others time: 0.0050470829010009766
predictions time: 0.6563045978546143
others time: 0.004833698272705078
predictions time: 0.657548189163208
others time: 0.005303621292114258
predictions time: 0.6481480598449707
others time: 0.00683903694152832
predictions time: 0.6582152843475342
others time: 0.005807399749755859
predictions time: 0.6538500785827637
others time: 0.005560398101806641
predictions time: 0.6554925441741943
others time: 0.005369901657104492
predictions time: 0.6386280059814453
others time: 0.004794120788574219
predictions time: 0.6143608093261719
others time: 0.005173444747924805
predictions time: 0.6227796077728271
others time: 0.00570368766784668
predictions time: 0.6461598873138428
others time: 0.005300045013427734
predictions time: 0.6602671146392822
others time: 0.005071878433227539
predictions time: 0.937431812286377
others time: 0.00533604621887207
predictions time: 0.6460938453674316
others time: 0.005266904830932617
prediction

In [41]:
output = predict('how s the weather today .')
output = predict('May I ask you a favor ?')
output = predict('bye')

predictions time: 0.634620189666748
others time: 0.005126237869262695
predictions time: 0.6374859809875488
others time: 0.004746913909912109
predictions time: 0.6320114135742188
others time: 0.0052928924560546875
predictions time: 0.9787411689758301
others time: 0.0053255558013916016
predictions time: 0.6324326992034912
others time: 0.005323886871337891
predictions time: 0.6491189002990723
others time: 0.005151033401489258
predictions time: 0.6095895767211914
others time: 0.00540924072265625
predictions time: 0.6279106140136719
others time: 0.005545854568481445
predictions time: 0.6340477466583252
others time: 0.005110502243041992
predictions time: 0.6544733047485352
others time: 0.005346059799194336
predictions time: 0.6550054550170898
others time: 0.00557255744934082
predictions time: 0.6481413841247559
others time: 0.005658626556396484
predictions time: 0.656029224395752
others time: 0.005881071090698242
predictions time: 0.6531870365142822
others time: 0.005446195602416992
predicti

In [0]:
# output = predict("It's a trap")

In [0]:
# # feed the model with its previous output
# sentence = 'I am not crazy, my mother had me tested.'
# for _ in range(5):
#   sentence = predict(sentence)
#   print('')

In [0]:
model.save_weights('/content/drive/My Drive/translate/transformer_translate_cn_weights.h5')
# model.save('/content/drive/My Drive/gdrive/transformer_chatbot_cn.h5')
tokenizer_resource.save_to_file("resource_vocab")
tokenizer_dest.save_to_file("dest_vocab")

In [47]:
del model
model = transformer(
    vocab_size_resource=VOCAB_SIZE_RESOURCE,
    vocab_size_dest=VOCAB_SIZE_DEST,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)
model.load_weights("/content/drive/My Drive/translate/transformer_translate_cn_weights.h5")
tokenizer_resource.load_from_file("resource_vocab")
tokenizer_dest.load_from_file("dest_vocab")
output = predict('hi , long time no see .')

predictions time: 0.5956630706787109
others time: 0.007515430450439453
predictions time: 0.600454568862915
others time: 0.005198955535888672
predictions time: 0.6065874099731445
others time: 0.0050716400146484375
predictions time: 0.6189320087432861
others time: 0.004744052886962891
predictions time: 0.6110286712646484
others time: 0.005135059356689453
predictions time: 0.609612226486206
others time: 0.0055484771728515625
predictions time: 1.1526410579681396
others time: 0.005269765853881836
predictions time: 0.618891716003418
others time: 0.005459308624267578
predictions time: 0.6170616149902344
others time: 0.005332231521606445
predictions time: 0.6296944618225098
others time: 0.005694866180419922
predictions time: 0.6254332065582275
others time: 0.005384206771850586
predictions time: 0.6213409900665283
others time: 0.0051746368408203125
predictions time: 0.6096799373626709
others time: 0.0053119659423828125
predictions time: 0.6202230453491211
others time: 0.005181074142456055
predi

## Summary

Here we are, we have implemented a Transformer in TensorFlow 2.0 in around 500 lines of code.

In this tutorial, we focus on the two different approaches to implement complex models with Functional API and Model subclassing, and how to incorporate them.

Try using a different dataset or hyper-parameters to train the Transformer! Thanks for reading.
